In [39]:
import onnx
import torch
import torch.nn as nn

import torch.nn.functional as F


class MinimalModel(nn.Module):
    def __init__(self):
        super(MinimalModel, self).__init__()
        # self.constant_zero_pad = nn.ConstantPad2d((1, 0, 0, 0), 0)
        # self.constant_zero_pad = nn.ZeroPad2d((1, 0, 0, 0))

    def forward(self, input_tensor):
        # return self.constant_zero_pad(input_tensor)
        return F.pad(input_tensor, (1,0,0,0), "constant", 0)

minimal_model = MinimalModel().cuda().eval()

# Random deep feature
input_tensor = torch.rand((1, 32, 128, 128))
# Check model can do a forward pass
print(minimal_model(input_tensor).shape)
# Export to onnx
torch.onnx.export(
    minimal_model,
    (input_tensor),
    'Fpad.onnx',
    do_constant_folding=True,
    export_params=True,
    verbose=False,
    training=False,
    opset_version=11
)
    
original_model = onnx.load('Fpad.onnx')
onnx.checker.check_model(original_model)

!trtexec --explicitBatch --onnx=Fpad.onnx --verbose



torch.Size([1, 32, 128, 128])
&&&& RUNNING TensorRT.trtexec # trtexec --explicitBatch --onnx=Fpad.onnx --verbose
[08/19/2021-17:57:31] [I] === Model Options ===
[08/19/2021-17:57:31] [I] Format: ONNX
[08/19/2021-17:57:31] [I] Model: Fpad.onnx
[08/19/2021-17:57:31] [I] Output:
[08/19/2021-17:57:31] [I] === Build Options ===
[08/19/2021-17:57:31] [I] Max batch: explicit
[08/19/2021-17:57:31] [I] Workspace: 16 MB
[08/19/2021-17:57:31] [I] minTiming: 1
[08/19/2021-17:57:31] [I] avgTiming: 8
[08/19/2021-17:57:31] [I] Precision: FP32
[08/19/2021-17:57:31] [I] Calibration: 
[08/19/2021-17:57:31] [I] Safe mode: Disabled
[08/19/2021-17:57:31] [I] Save engine: 
[08/19/2021-17:57:31] [I] Load engine: 
[08/19/2021-17:57:31] [I] Builder Cache: Enabled
[08/19/2021-17:57:31] [I] NVTX verbosity: 0
[08/19/2021-17:57:31] [I] Inputs format: fp32:CHW
[08/19/2021-17:57:31] [I] Outputs format: fp32:CHW
[08/19/2021-17:57:31] [I] Input build shapes: model
[08/19/2021-17:57:31] [I] Input calibration shapes: mo

/opt/conda/lib/python3.6/site-packages/torch/onnx/utils.py:284: UserWarning: It is recommended that constant folding be turned off ('do_constant_folding=False') when exporting the model in training-amenable mode, i.e. with 'training=TrainingMode.TRAIN' or 'training=TrainingMode.PRESERVE' (when model is in training mode). Otherwise, some learnable model parameters may not translate correctly in the exported ONNX model because constant folding mutates model parameters. Please consider turning off constant folding or setting the training=TrainingMode.EVAL.
  warnings.warn("It is recommended that constant folding be turned off ('do_constant_folding=False') "


----------------------------------------------------------------
Input filename:   Fpad.onnx
ONNX IR version:  0.0.6
Opset version:    11
Producer name:    pytorch
Producer version: 1.7
Domain:           
Model version:    0
Doc string:       
----------------------------------------------------------------
[08/19/2021-17:57:38] [V] [TRT] Plugin creator already registered - ::BatchTilePlugin_TRT version 1
[08/19/2021-17:57:38] [V] [TRT] Plugin creator already registered - ::BatchedNMS_TRT version 1
[08/19/2021-17:57:38] [V] [TRT] Plugin creator already registered - ::CoordConvAC version 1
[08/19/2021-17:57:38] [V] [TRT] Plugin creator already registered - ::CropAndResize version 1
[08/19/2021-17:57:38] [V] [TRT] Plugin creator already registered - ::DetectionLayer_TRT version 1
[08/19/2021-17:57:38] [V] [TRT] Plugin creator already registered - ::FlattenConcat_TRT version 1
[08/19/2021-17:57:38] [V] [TRT] Plugin creator already registered - ::GenerateDetection_TRT version 1
[08/19/2021